In [1]:
import numpy as np

from tensorflow import keras

import fedhex as fx
from fedhex.pretrain import generation as fxgen
from fedhex.train import SelectiveProgbarLogger, BatchLossHistory
from fedhex.train.tf import loss_MADE

2024-06-07 11:58:49.189630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Welcome to JupyROOT 6.30/02


In [2]:
ndim = 2
nx = ny = 10
sigmax = 1 / (nx + 1) / 2
sigmay = 1 / (ny + 1) / 2
sigma = np.array([sigmax, sigmay])

gen = fx.GridGaussGenerator(cov_strat=fxgen.RepeatStrategy(fxgen.DiagCov(ndim=ndim, sigma=sigma)),
                      ngausx=nx,
                      ngausy=ny,
                      seed=0x2024)

nsamp = 1000
samp, lab = gen.generate(nsamp=nsamp)
lablocs = np.unique(lab, axis=0)

In [3]:
nmade = 8
ninputs = ncinputs = ndim
hidden_units = [16, 16]
batch_size = 1 << 12
end_epoch = 20


log_freq = 10
flow_path = "model/2024.06.07-gg"
callbacks = []
callbacks.append(SelectiveProgbarLogger(1, epoch_interval=log_freq, epoch_end=end_epoch))
callbacks.append(BatchLossHistory(flow_path + "/loss.npy"))


model = fx.MADEManager(nmade=nmade,
               ninputs=ninputs,
               ncinputs=ncinputs,
               hidden_units=hidden_units,
               activation="relu")

opt = keras.optimizers.Adam(learning_rate=1e-3)
loss = loss_MADE

model.compile_model(opt=opt, loss=loss)

model.train_model(dm=gen,
                  batch_size=batch_size,
                  end_epoch=end_epoch,
                  callbacks=callbacks,
                  path=flow_path)

11:59:02.33 <I> [Training begins]
epoch begin: 2024-06-07 11:59:21.955074 | time elapsed: 0:00:19.467556
Epoch 10/20
25/25 [==============================] - 1s 22ms/sample - loss: -0.2613
epoch begin: 2024-06-07 11:59:28.179261 | time elapsed: 0:00:25.691743
Epoch 20/20
25/25 [==============================] - 1s 26ms/sample - loss: -0.5747
INFO:tensorflow:Assets written to: model/2024.06.07-gg/assets


INFO:tensorflow:Assets written to: model/2024.06.07-gg/assets


11:59:41.76 <I> [Training ends]
0:00:39.43 <I> [Time elapsed]


/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'function'>, which may lead to improper serialization.
  warnings.warn(
/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'function'>, which may lead to improper serialization.
  warnings.warn(


In [4]:
mm = fx.MADEManager.import_model(flow_path)

In [5]:
end_epoch = int(2 * end_epoch)
callbacks = []
callbacks.append(SelectiveProgbarLogger(1, epoch_interval=log_freq, epoch_end=end_epoch))
callbacks.append(BatchLossHistory(flow_path + "/loss.npy"))
mm.train_model(dm=gen,
                  batch_size=batch_size,
                  starting_epoch=int(end_epoch/2),
                  end_epoch=end_epoch,
                  callbacks=callbacks,
                  path=flow_path)

11:59:49.67 <I> [Training begins]
epoch begin: 2024-06-07 12:00:07.698709 | time elapsed: 0:00:17.912309
Epoch 30/40
25/25 [==============================] - 1s 27ms/sample - loss: -0.6462
epoch begin: 2024-06-07 12:00:13.842789 | time elapsed: 0:00:24.056389
Epoch 40/40
25/25 [==============================] - 1s 23ms/sample - loss: -0.6661
INFO:tensorflow:Assets written to: model/2024.06.07-gg/assets


INFO:tensorflow:Assets written to: model/2024.06.07-gg/assets


12:00:27.59 <I> [Training ends]
0:00:37.91 <I> [Time elapsed]


/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'function'>, which may lead to improper serialization.
  warnings.warn(
/home/asatk/miniconda3/envs/rootflows/lib/python3.11/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'function'>, which may lead to improper serialization.
  warnings.warn(
